In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model

model_path = '/content/drive/MyDrive/real_vs_fake_final.h5'
model = load_model(model_path)

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import Image

# Preprocessing function
def preprocess_and_predict(img: Image.Image):
    img = img.resize((224, 224))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    label = "FAKE" if prediction < 0.9 else "REAL"
    confidence = float(prediction) if prediction > 0.5 else 1 - float(prediction)
    return f"{label} ({confidence * 100:.2f}% confidence)"

interface = gr.Interface(
    fn=preprocess_and_predict,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Real vs Fake Image Classifier",
    description="Upload an image to check if it's REAL or FAKE.",
    theme="default"
)

interface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8a9e7ce932e114eb73.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [ ]:
import cv2

def extract_frames(video_path, frame_rate=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    success = True
    while success:
        success, frame = cap.read()
        if count % frame_rate == 0 and success:
            frames.append(frame)
        count += 1
    cap.release()
    return frames


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def predict_frames(frames, model):
    results = []
    for frame in frames:
        img = cv2.resize(frame, (224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array, verbose=0)[0][0]
        results.append(prediction)
    return results

In [ ]:
def classify_video(predictions, threshold=0.9):
    avg_score = np.mean(predictions)
    print(f"Average Confidence Score: {avg_score:.4f}")
    return "REAL" if avg_score > threshold else "FAKE"

In [ ]:
from google.colab import files
uploaded = files.upload()

import os
video_path = next(iter(uploaded))


Saving 01_02__exit_phone_room__YVGY8LOK.mp4 to 01_02__exit_phone_room__YVGY8LOK.mp4


In [ ]:
frames = extract_frames(video_path, frame_rate=10)
print(f"Extracted {len(frames)} frames from video.")

predictions = predict_frames(frames, model)
final_decision = classify_video(predictions)

print(f"✅ Final Video Prediction: {final_decision}")

Extracted 21 frames from video.
Average Confidence Score: 0.8900
✅ Final Video Prediction: FAKE


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image

def extract_frames_from_video(video_path, frame_rate=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0
    success, frame = cap.read()

    while success:
        if frame_count % frame_rate == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            frames.append(pil_img)
        success, frame = cap.read()
        frame_count += 1

    cap.release()
    return frames

def predict_frames(frames, model):
    predictions = []
    for img in frames:
        img = img.resize((224, 224))
        img_array = image.img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)[0][0]
        predictions.append(pred)
    return predictions

def classify_video(predictions, threshold=0.5):
    avg_score = np.mean(predictions)
    print(f"Average Confidence Score: {avg_score:.4f}")
    return "REAL" if avg_score > threshold else "FAKE"


In [ ]:
def process_video(video_file):
    frames = extract_frames_from_video(video_file, frame_rate=10)
    if not frames:
        return "❌ Could not extract frames from video."

    predictions = predict_frames(frames, model)
    return classify_video(predictions)


In [ ]:
import gradio as gr
from tensorflow.keras.models import load_model

model = load_model("/content/drive/MyDrive/real_vs_fake_final.h5")

interface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),
    outputs="text",
    title="Real vs Fake Video Classifier",
    description="Upload a video to check if it contains REAL or FAKE faces."
)

interface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7040cc9e7fe2384534.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━

ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/fastapi/applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/applications.py", line 112, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py", line 187, in __call__
    raise exc
  File "/usr/local/lib/python3.11/dist-packages/starlette/middleware/errors.py",